# Notebook for making animation of the distributions while changing parameters

In [1]:
import numpy as np
# from tqdm import tqdm
from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt
import numba
from numba import prange
from time import perf_counter
from scipy.ndimage import convolve, sobel
from scipy import ndimage
from scipy.special import xlogy
from sklearn import preprocessing
from scipy.stats import mode
from scipy.stats import gaussian_kde
from scipy.integrate import quad
import seaborn as sns
from IPython import display
import time
from matplotlib.animation import FuncAnimation

# import statistics as statss
%matplotlib inline
sns.set_style("ticks")
sns.set_context("poster")

def timer(method):
    def timed(*args, **kw):
        ts = perf_counter()
        result = method(*args, **kw)
        te = perf_counter()
        tlapsed = te-ts
        print(f"{tlapsed//60} mins and {tlapsed%60:0.2f} secs")
        return result
    return timed

Functions for simulating the Gray-Scott model:

In [2]:
s = [[1,1,1],
     [1,1,1],
     [1,1,1]]

@numba.njit(fastmath=True, parallel=True)
def gray_scott(U, V, Du, Dv, f, k, dt, dx, T):
    """
    Integrates the gray-scott system over time using the given initial
    conditions.
    """
    n = U.shape[0]
    iters = int(T / dt)
    for i in range(iters):
        Lu, Lv = Du*lap(U, n)/dx/dx, Dv*lap(V, n)/dx/dx
        U, V = U + dt*(Lu - U*V*V + f * (1-U)), V + dt*(Lv + U*V*V - (f+k) * V)
    return U, V

def gscott(n, F, k, T, Du=0.16, Dv=0.08, dt=1.0, dx=1.0, seed=5000000):
    """
    Wrapper function that initializes the U and V concentration arrays and the
    random number generator.
        
        n: dimensions of the discretized system
        F: value of F used for the model
        k: value of k used for the model
        T: number of time steps simulated
    
    """
    np.random.seed(seed=seed)
    U, V = np.zeros((n, n), dtype=np.float64), np.zeros((n, n), dtype=np.float64)
    r, m = n//40, n//2
    U[...] = 1.0
    V[m-r:m+r, m-r:m+r] = 0.25
    U[m-r:m+r, m-r:m+r] = 0.5
    U += np.random.normal(scale=0.05, size=U.shape)
    V += np.random.normal(scale=0.05, size=V.shape)
    return gray_scott(U, V, Du, Dv, F, k, dt, dx, T)

def calc_objects(inp):
    """
    Calculates the number and size of objects in a given array.
    
    An array is returned with the size of each object, the length of the 
    array thus being the number of objects. A [0] array is returned if
    no objects are found.
    """
    fftding = np.fft.fft2(inp)
    outp = ndimage.fourier_ellipsoid(fftding, 1.1)*100
    outp = np.fft.ifft2(ndimage.fourier_gaussian(outp, 1.01)).real*10

    binarized1 = np.clip(outp-((outp.min() + outp.max())/2), 0, 1)
    labels1 = ndimage.label(binarized1, structure=s)
    binarized2 = np.clip((outp-((outp.min() + outp.max())/2))*-1, 0, 1)
    labels2 = ndimage.label(binarized2, structure=s)
    if labels1[1] > labels2[1]:
        bins, edges = np.histogram(labels1[0], bins=labels1[1])
        return bins[1:]

    # Try inversed region
    if labels2[1] > 1:
        bins, edges = np.histogram(labels2[0], bins=labels2[1])
        return bins[1:]
    
    # No objects
    return np.zeros(1)

@numba.njit(parallel=True, fastmath=True)
def lap(u, N):
    """
    Parallel implementation of the laplacian operator with periodic boundary conditions.
    """
    uc = np.empty((N, N))
    for x in numba.prange(1, N-1):
        uc[x, 1:-1] = u[x+1, 1:-1] + u[x-1, 1:-1] + u[x, :-2] + u[x, 2:] - 4*u[x, 1:-1]

    uc[1:-1, 0] = u[1:-1, 1] + u[:-2, 0] + u[2:, 0] + u[1:-1, -1] - 4*u[1:-1, 0]
    uc[1:-1, -1] = u[1:-1, -2] + u[:-2, -1] + u[2:, -1] + u[1:-1, 0] - 4*u[1:-1, -1]
    uc[0, 1:-1] = u[1, 1:-1] + u[0, 2:] + u[0, :-2] + u[-1, 1:-1] - 4*u[0, 1:-1]
    uc[-1, 1:-1] = u[-2, 1:-1] + u[-1, 2:] + u[-1, :-2] + u[0, 1:-1] - 4*u[-1, 1:-1]

    uc[0, 0] = u[0, 1] + u[0, -1] + u[1, 0] + u[-1, 0] - 4*u[0, 0]
    uc[-1, 0] = u[0, 0] + u[-2, 0] + u[-1, -1] + u[-1, -2] - 4*u[-1, 0]
    uc[0, -1] = u[0, 0] + u[0, -2] + u[-1, -1] + u[1, -1] - 4*u[0, -1]
    uc[-1, -1] = u[0, -1] + u[-1, 0] + u[-1, -2] + u[-2, -1] - 4*u[-1, -1]
    return uc

## Code for animation

The following functions are used for making the animation:

In [7]:
def bulk_gaussian(f, k, n=150, T=10000):
    """
    Runs the Gray-Scott model simulation for each parameter setting returning a
    list of all object lists resulted from the simulations.
    """
    kdes = [None for j in range(len(f))]
    positions = np.linspace(0, 200, 1000)
    for index in tqdm(range(len(f))):
        u, v = gscott(n, f[index], k[index], T)
        obj_u = calc_objects(u)
        kdes[index] = obj_u
    return kdes

def animate_pdf(f, k, n=200, T=20000):
    """
    For each parameter setting, the distribution of component sizes is plotted.
    The figures are saved which can be used to make a animation.
    """
    pdfs = bulk_gaussian(f, k, n, T) 
    for i in range(len(pdfs)):
        fig, ax1 = plt.subplots()  
        plt.title("{}".format(i))
        sns.distplot(pdfs[i], ax=ax1)
        ax1.set_xlim(left=0)
        ax1.set_ylim(0, 0.25)
        fig.savefig("Pics/pdfs/{}".format(i))
        plt.close()

Setting the parameter ranges and starting the simulation:

In [8]:
# These are the parameter changes when changing from stable dots to chaotic dots:
# f = np.linspace(0.0238191, 0.0271360, 30)
# k = np.linspace(0.05869347, 0.06141, 30)

# These are the parameter changes when changing from stable dots to snake patterns:
f = np.linspace(0.0395, 0.0428, 30)
k = np.linspace(0.0652, 0.0632, 30)

kdes = animate_pdf(f, k)